In [22]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import time
import torch.nn.functional as F
from matplotlib import pyplot as plt
from PIL import Image
import d2l
import imageio

import numpy as np
from measure import SegmentationMetric
from dataset import load_dataset   # from xx.py文件 import里面的函数
from MACUNet import MACUNet
from early_stopping import EarlyStopping
from tqdm import tqdm, trange
import glob

import cv2
from matplotlib import pyplot as plt 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
torch.backends.cudnn.enabled =True
# torch.backends.cudnn.benchmark = True
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
#一些参数的设置
batch_size = 64 #batch_size
niter = 50    #迭代次数,epoches
class_num = 7   #类别数 背景-1、建筑-2、道路-3、水-4、贫瘠-5、森林-6、农业-7。没有数据的区域被指定为0，应该忽略 
learning_rate = 0.0001 * 3  #学习率
beta1 = 0.5     #beta1,应该也是Adam中的？
cuda = True
num_workers = 12 #读入数据时候的线程
size_h = 256   #输入的images高度
size_w = 256   #输入的images宽度
flip = 0       #翻转
band = 3       #通道数，RGB则为3,遥感图像还有其他多光谱的
net = MACUNet(band, class_num)   #MACUNET网络

################全部数据################
# train_path = '/root/master/ATL/MACU-Net-master/data/LoveDA/Train/Urban' #训练集地址
# val_path = '/root/master/ATL/MACU-Net-master/data/LoveDA/Val/Urban'     #验证集地址
# test_path ='/root/master/ATL/MACU-Net-master/data/LoveDA/Test/Urban'    #测试集地址

train_path = '/root/master/ATL/dataset/LoveDA_256/Train/Urban' #训练集地址
val_path = '/root/master/ATL/dataset/LoveDA_1024/Val/Urban'     #验证集地址
test_path ='/root/master/ATL/dataset/LoveDA_1024/Val/Urban' 

###############小数据测试####################
# train_path = '/root/master/ATL/dataset/render/little_data/Train/Urban' #训练集地址
# val_path = '/root/master/ATL/dataset/render/little_data/Val/Urban'     #验证集地址
# test_path ='/root/master/ATL/dataset/render/little_data/Test/Urban'    #测试集地址

out_file = '../checkpoint/' + net.name  #输出的地址

num_GPU = 1  #GPU的个数
index = 640
# torch.cuda.set_device(0) #设置设备为GPU0？？？？？如果是多个GPU呢？？？
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
train_path = '/root/master/ATL/dataset/LoveDA_256/Train/Urban' #训练集地址
val_path = '/root/master/ATL/dataset/LoveDA_256/Val/Urban'     #验证集地址
test_path ='/root/master/ATL/dataset/LoveDA_256/Val/Urban' 

In [4]:
################加载训练集和验证集############
train_dataset_ = load_dataset(train_path, size_w, size_h, flip, band, batch_size) #训练集
val_dataset_ = load_dataset(val_path, size_w, size_h, 0, band) #验证集


print('-----------------------------------------')   
if train_dataset_.src_list != []:
    print(f'  -训练集数据加载成功，共{len(train_dataset_.src_list)}条')
    print('  -训练集images shape',train_dataset_.__getitem__(0)[0].shape)
    print('  -训练集label shape',train_dataset_.__getitem__(0)[1].shape)
    
else:
    print('  --训练集数据加载失败--')
    
print('-----------------------------------------')    
if val_dataset_.src_list != []:
    print(f'  -测试集数据加载成功，共{len(val_dataset_.src_list)}条')
    print('  -验证集images shape',val_dataset_.__getitem__(0)[0].shape)
    print('  -验证集label shape',val_dataset_.__getitem__(0)[1].shape)
else:
    print('  --测试集数据加载失败--')
print('-----------------------------------------')    

-----------------------------------------
  -训练集数据加载成功，共36992条
  -训练集images shape (256, 256, 3)
  -训练集label shape (256, 256)
-----------------------------------------
  -测试集数据加载成功，共21664条
  -验证集images shape (256, 256, 3)
  -验证集label shape (256, 256)
-----------------------------------------


In [19]:
src_list = np.array(sorted(glob.glob(train_path + '/images_png/' + '*.png'))) #把

In [27]:
start = time.time()
for i in range(len(src_list)):
    img = np.array(Image.open(src_list[i]))
    del img
end = time.time()
print(f'共耗时{end-start}秒')

共耗时187.81041526794434秒
